In [1]:
from surrealengine import (
    Document, RelationDocument, StringField, IntField, FloatField,
    BooleanField, DateTimeField, create_connection
)

In [2]:
# Define a Document class for Person
class Person(Document):
    """A document representing a person."""
    name = StringField(required=True)
    age = IntField(min_value=0)
    email = StringField()

    class Meta:
        collection = "people"


# Define a Document class for Book
class Book(Document):
    """A document representing a book."""
    title = StringField(required=True)
    isbn = StringField()
    published_year = IntField()
    price = FloatField(min_value=0)

    class Meta:
        collection = "books"


# Define a RelationDocument class for AuthorRelation
class AuthorRelation(RelationDocument):
    """A relation document representing an author relationship."""
    date_written = DateTimeField()
    is_primary_author = BooleanField(default=True)

    class Meta:
        collection = "authored"

In [3]:
connection = create_connection(
        url="ws://db:8000/rpc",
        namespace="test_ns",
        database="test_db",
        username="root",
        password="root",
        make_default=True
    )

await connection.connect()
print("Connected to SurrealDB")

Connected to SurrealDB


In [4]:
# Create tables
await Person.create_table(connection)
await Book.create_table(connection)

# Create a person with an embedded address
person = Person(
    name="John Doe",
    age=35,
    email="john.doe@example.com"
)
await person.save()
print(f"Created person: {person.to_dict()}")

# Create a book
book = Book(
    title="The Great Novel",
    isbn="978-3-16-148410-0",
    published_year=2023,
    price=19.99
)
await book.save()
print(f"Created book: {book.to_dict()}")

Created person: {'name': 'John Doe', 'age': 35, 'email': 'john.doe@example.com', 'id': 'people:rz68u0uo909f3ticnmj5'}
Created book: {'title': 'The Great Novel', 'isbn': '978-3-16-148410-0', 'published_year': 2023, 'price': 19.99, 'id': 'books:byrfrh9vr13hz0zqt812'}


In [5]:
relation = await AuthorRelation.create_relation(
            person, book,
            date_written="2022-01-15T00:00:00Z",
            is_primary_author=True
        )

In [6]:
await person.fetch_relation('authored')

{'id': RecordID(table_name=people, record_id='rz68u0uo909f3ticnmj5'),
 'related': [RecordID(table_name=books, record_id='byrfrh9vr13hz0zqt812')]}

In [7]:
await person.resolve_relation('authored')

[{'id': RecordID(table_name=books, record_id='byrfrh9vr13hz0zqt812'),
  'isbn': '978-3-16-148410-0',
  'price': 19.99,
  'published_year': 2023,
  'title': 'The Great Novel'}]

In [8]:
# Clean up
await person.delete()
await book.delete()
print("Deleted person and book")

Deleted person and book
